In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('master_dataset_clean.csv')

In [3]:
df = df.drop(["County", "State", "Unnamed: 0"], axis=1)
from scipy import stats
import numpy as np
#drop all upper bound outliers
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]
#drop all lower bound outliers
df = df[(np.abs(stats.zscore(df)) > -3).all(axis=1)]
#drop zero row values
df = df.loc[(df!=0).any(axis=1)]
print(len(df))
df.describe()

2521


,FIPS,live 2a1w1c,min 1a1c,pct_less_hs,pct_hs,pct_some_college,pct_college,crime_per_10k_x,unemployment_rate_2019,median_household_income_2019,...,Civic,Other,Required annual income after taxes,Annual taxes,Required annual income before taxes,crime_per_10k_y,violent_crime_per_10k,hard_drugs_crime_per_10k,soft_drugs_crime_per_10k,commerce_crimes_per_10k
count,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,...,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000,2521.000000
mean,29615.689012,26.701376,8.199976,13.018247,34.962793,31.360928,20.659104,192.026743,3.839746,53915.084887,...,3667.780246,5097.151924,44939.614835,10604.896470,55544.431575,192.026818,35.840652,6.670541,27.873544,24.111956
std,14904.045836,1.323503,1.313342,5.606045,6.501269,4.823432,7.754161,156.926734,1.174521,11552.462166,...,139.195298,50.456986,1957.014138,1921.424454,2741.339645,156.926645,28.023552,8.919255,26.437072,22.125754
min,1001.000000,23.450000,7.250000,1.100000,15.200000,15.800000,7.200000,0.000000,1.400000,24732.000000,...,3554.000000,4996.000000,42127.000000,6972.000000,49979.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18115.000000,25.790000,7.250000,8.700000,30.700000,28.000000,15.100000,68.100000,3.000000,45939.000000,...,3554.000000,5070.000000,43661.000000,9517.000000,53676.000000,68.096204,14.088716,0.000000,7.280140,5.952008
50%,28141.000000,26.550000,7.250000,12.000000,35.200000,31.300000,19.000000,164.900000,3.600000,52554.000000,...,3554.000000,5070.000000,44395.000000,10766.000000,55219.000000,164.901664,31.864987,3.261997,21.629755,18.477458
75%,41047.000000,27.390000,9.250000,16.700000,39.500000,34.600000,24.200000,282.400000,4.500000,60384.000000,...,3838.000000,5144.000000,45672.000000,11805.000000,56978.000000,282.396190,52.732233,9.033424,41.008856,37.201281
max,56045.000000,32.730000,12.000000,31.800000,54.500000,46.300000,50.400000,787.370000,8.200000,98217.000000,...,3838.000000,5144.000000,53719.000000,17831.000000,68082.000000,787.373150,152.596116,51.091500,161.049063,102.758791


In [4]:
from sklearn.linear_model import LinearRegression

In [5]:
df.columns

Index(['FIPS', 'live 2a1w1c', 'min 1a1c', 'pct_less_hs', 'pct_hs',
       'pct_some_college', 'pct_college', 'crime_per_10k_x',
       'unemployment_rate_2019', 'median_household_income_2019', 'Food',
       'Medical', 'Housing', 'Transportation', 'Civic', 'Other',
       'Required annual income after taxes', 'Annual taxes',
       'Required annual income before taxes', 'crime_per_10k_y',
       'violent_crime_per_10k', 'hard_drugs_crime_per_10k',
       'soft_drugs_crime_per_10k', 'commerce_crimes_per_10k'],
      dtype='object')

In [15]:
df_X = df.drop(["FIPS", 'crime_per_10k_x','crime_per_10k_y','violent_crime_per_10k', 'hard_drugs_crime_per_10k','soft_drugs_crime_per_10k', 'commerce_crimes_per_10k'], axis=1)
y = df['crime_per_10k_y']

In [33]:
from itertools import chain, combinations
def all_subsets(ss):
    return chain(*map(lambda x: combinations(ss, x), range(1, len(ss)-5)))
from sklearn.model_selection import train_test_split

best = 0.0011602423936243
for subset in all_subsets(df_X.columns):
    X = df_X[list(subset)]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    r2 = lm.score(X_test, y_test)
    if r2 > best:
        best = r2
        best_subset = subset

print(best_subset)
print(best)
print(df_X.columns)


('min 1a1c', 'Food', 'Medical', 'Housing', 'Transportation', 'Civic', 'Required annual income after taxes', 'Annual taxes', 'Required annual income before taxes')
0.08669010433877355
Index(['live 2a1w1c', 'min 1a1c', 'pct_less_hs', 'pct_hs', 'pct_some_college',
       'pct_college', 'unemployment_rate_2019', 'median_household_income_2019',
       'Food', 'Medical', 'Housing', 'Transportation', 'Civic', 'Other',
       'Required annual income after taxes', 'Annual taxes',
       'Required annual income before taxes'],
      dtype='object')
